# In this demo, we walk through how to retrieve and use our data

As described in the paper, we discovered press release/article pairs through two primary directions: __forwards__ (i.e. crawling news articles and discovering hyperlinks to press releases) and __backwards__ (i.e. crawling press releases and querying a backlink service to discover news articles.) 

We show how to access the data collected from each direction now.

## Forwards Direction: Press Releases $\leftarrow$ News Articles (Hyperlinks)

The data for this direction is primarily stored in a SQLite3 database: `data/article_to_press_release_data.db.tar.gz`

It can be accessed from this [Google Drive link](https://drive.google.com/drive/folders/1CmJkwpbV84pYaEMtNWhWkzG-B4_3hXR8?usp=sharing).

## Backwards Direction: Press Releases $\rightarrow$ News Articles  (Backlinks)

The data for this direction is primarily stored in three files:

They can be accessed from this [Google Drive Link](https://drive.google.com/drive/folders/11IpwmFKuFn7LryUHW1df1fcJ2RmFUub1?usp=drive_link).

In [ ]:
from datasets import load_from_disk
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
data_dir = '../data'
source_df = pd.read_json(f'{data_dir}/full-source-scored-data.jsonl.gz', lines=True, compression='gzip', nrows=5000)
article_d = load_from_disk(f'{data_dir}/all-coref-resolved')

In [7]:
article_d

Dataset({
    features: ['article_url', 'target_timestamp_key', 'target_timestamp', 'sort_criteria', 'wayback_url', 'wayback_timestamp', 'method', 'links', 'article_text', 'word_lists', 'sent_lists', 'best_class', 'coref_resolved_sents'],
    num_rows: 496380
})

In [ ]:
a_urls = set(source_df['article_url'])
filtered_article_d = article_d.filter(lambda x: x['article_url'] in a_urls, num_proc=10)
filtered_article_df = filtered_article_d.to_pandas()

In [49]:
disallowed_quote_types = set(['Other', 'Background/Narrative', 'No Quote'])

sentences_with_quotes = (
    filtered_article_d
         .to_pandas()
         .merge(source_df, on='article_url')
         [['article_url', 'attributions', 'quote_type', 'sent_lists',]]
         .explode(['attributions', 'quote_type', 'sent_lists'])
)

sentences_with_quotes = (sentences_with_quotes
     .assign(attributions=lambda df: 
             df.apply(lambda x: x['attributions'] if x['quote_type'] not in disallowed_quote_types else np.nan, axis=1)
    )
)

In [ ]:
one_article = (
    sentences_with_quotes
         .loc[lambda df: df['article_url'] == df['article_url'].unique()[1]]
        .reset_index(drop=True)
)

doc_str = one_article[['sent_lists', 'attributions']].to_csv(sep='\t', index=False)
json_str = one_article[['sent_lists', 'attributions']].to_json(lines=True, orient='records')

# Finding Contrasting Press Releases

In our paper, we described a method to find news articles that provided "critical coverage" of press releases. This method used aggregated sentence-level NLI scores on the document level.

![alt text](images/doc-level-nli.png "Title")

for more information, see `src/assess_factual_consistency.py`. However, we do recommend taking a different, more modern approach, possibly one that utilizes LLMs to distill a BERT-based classification model.

# Extracting Sources

I ran llama3 locally using a cool program called Ollama: https://ollama.com/. You can follow the link to install the program.

This can be useful for trying things out before we work out compute access.

Run this in your terminal:

`ollama run llama3`

Even llama3 8b is pretty powerful. Llama 70b is better, but that may not run on your local computer.

In [111]:
import requests
import pyperclip

In [ ]:
pyperclip.copy(doc_str)
pyperclip.copy(json_str)

In [105]:
r = requests.post(
    'http://localhost:11434/api/generate', 
    json = {
        "model": "llama3",
        "prompt":f"""
            Here is a news article, with each sentence annotated according to the source of it's information:
            ```
            {json_str}
            ```

            Please summarize each of our source annotations. Tell me in one paragraph per source: (1) who the source is (2) what informational content they provide to the article. 
            Only rely on the annotations I have provided, don't identify additional sources.
        """,
        "stream": False
})

In [106]:
print(r.json()['response'])

**Socrata Foundation**: The Socrata Foundation is a 501(c)(3) organization that provides information about its philanthropic philosophy and mission. They describe their support for unique organizations that lack resources or financial means to fulfill their data-driven missions. They also mention the importance of open data in removing barriers to social justice and economic progress.

**Robert Runge**: Robert Runge, a member of Socrata's Board of Directors, provides additional context about the Socrata Foundation's purpose and how it bridges the gap between publicly funded open data projects and underfunded or unfunded opportunities.

**Mike Duggan**: Detroit Mayor Mike Duggan shares his perspective on why he turned to data-driven government in Detroit. He highlights the importance of transparency, accountability, and fact-based decision-making enabled by open data. He also explains how the Socrata Foundation helped Detroit gain access to the necessary technology and infrastructure.

